# [Clustering4Ever](https://github.com/Clustering4Ever/Clustering4Ever) on [SparkNotebook](http://spark-notebook.io/) by [LIPN](https://lipn.univ-paris13.fr/) [A3](https://lipn.univ-paris13.fr/accueil/equipe/a3/) team

# Clusterwise PLS

In [ ]:
import clustering4ever.spark.clustering.clusterwise.{Clusterwise, ClusterwiseCore, PLS, ClusterwiseModel}
import scala.collection.{mutable, immutable}

import clustering4ever.spark.clustering.clusterwise.{Clusterwise, ClusterwiseCore, PLS, ClusterwiseModel}
import scala.collection.{mutable, immutable}


In [ ]:
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Clusterwise/simdata.csv

--2018-09-07 13:30:32--  http://www.clustering4ever.org/Datasets/Clusterwise/simdata.csv
Resolving www.clustering4ever.org (www.clustering4ever.org)... 62.210.16.62
Connecting to www.clustering4ever.org (www.clustering4ever.org)|62.210.16.62|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61840 (60K) [text/csv]
Saving to: ‘/tmp/simdata.csv.2’

     0K .......... .......... .......... .......... .......... 82% 8.96M 0s
    50K ..........                                            100%  124M=0.006s

2018-09-07 13:30:32 (10.7 MB/s) - ‘/tmp/simdata.csv.2’ saved [61840/61840]


import sys.process._


In [ ]:
val dataXY = scala.io.Source.fromFile("/tmp/simdata.csv")
  .getLines
  .toSeq
  .map(_.split(',').map(_.toDouble))
  .map( a => (mutable.ArrayBuffer(a.take(10):_*), mutable.ArrayBuffer(a.takeRight(3):_*)) )
  .zipWithIndex
  .map{ case(xy,idx) => (idx, xy) }
  .toBuffer

dataXY: scala.collection.mutable.Buffer[(Int, (scala.collection.mutable.ArrayBuffer[Double], scala.collection.mutable.ArrayBuffer[Double]))] = ArrayBuffer((0,(ArrayBuffer(4.058845733, 6.405834969, 4.943772245, 4.191393637, 5.691694039, 6.18161279, 6.182385889, 5.982450545, 6.139507172, 7.42310638),ArrayBuffer(6.508480628, 6.009293422, 6.61563233))), (1,(ArrayBuffer(5.102442757, 4.716437077, 4.692925255, 4.839302826, 4.02386069, 2.158591158, 2.901072862, 2.816463237, 3.501312504, 3.084936398),ArrayBuffer(5.238588818, 5.070952054, 5.370006441))), (2,(ArrayBuffer(4.766958169, 4.838583906, 6.485887384, 4.404467358, 5.211054103, 4.865929818, 5.278244008, 4.604447037, 4.98848043, 4.768183695),ArrayBuffer(5.917350113, 5.666997815, 6.045082662))), (3,(ArrayBuffer(6.138171718, 4.494158628, 3.410...

## Run a full Clusterwise
### Parameters

In [ ]:
val init = 20
val nbCV = 10
val g = 4
val h = 2
val sizeBloc = 10
val kPred = 25
val nbLaunch = 1
val standardization = true
val withY = false

init: Int = 20
nbCV: Int = 10
g: Int = 4
h: Int = 2
sizeBloc: Int = 10
kPred: Int = 25
nbLaunch: Int = 1
standardization: Boolean = true
withY: Boolean = false


In [ ]:
val t0 = System.currentTimeMillis
val (sqrmseCalAndVal, _) = Clusterwise.run(sc, dataXY, g, h, nbCV, init, kPred, withY, standardization, sizeBloc)
val t1 = System.currentTimeMillis

(t1 - t0) / 1000D

t0: Long = 1536319838701
sqrmseCalAndVal: Array[(Double, Double)] = Array((0.0940799385940823,33.00121233628397), (0.09191440434453503,31.61651985189719), (0.09529592245083816,25.552908614960923), (0.09812158183745749,29.45447129907458), (0.09305115739071786,20.36060545690047), (0.10427563455967383,31.828118845478865), (0.09183282617834347,28.824748501372937), (0.08497175134177248,33.90429422743793), (0.09274565080292248,29.151690569907533), (0.0871305914881403,23.402680781896617))
t1: Long = 1536319881294
res6: Double = 42.593


42.593

In [ ]:
sqrmseCalAndVal

res8: Array[(Double, Double)] = Array((0.0940799385940823,33.00121233628397), (0.09191440434453503,31.61651985189719), (0.09529592245083816,25.552908614960923), (0.09812158183745749,29.45447129907458), (0.09305115739071786,20.36060545690047), (0.10427563455967383,31.828118845478865), (0.09183282617834347,28.824748501372937), (0.08497175134177248,33.90429422743793), (0.09274565080292248,29.151690569907533), (0.0871305914881403,23.402680781896617))


entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total

## Run one Initialization of Clusterwise

In [ ]:
val (dsPerClass, predFitted, coIntercept, coXYcoef, critReg, mapsRegCrit, classedReg) = ClusterwiseCore.plsPerDot(dataXY, h, g)

dsPerClass: Array[Array[(Int, (scala.collection.mutable.ArrayBuffer[Double], scala.collection.mutable.ArrayBuffer[Double], Int))]] = Array(Array((7,(ArrayBuffer(6.685955917, 6.769605686, 5.342186991, 5.360624854, 5.205307736, 8.023073865, 8.646419433, 6.845253485, 7.939636587, 7.158224572),ArrayBuffer(6.857725732, 6.853841789, 6.627549623),0)), (23,(ArrayBuffer(3.334824219, 3.671672823, 3.644531857, 4.028140891, 3.844837271, 5.629685303, 7.897636537, 7.574256116, 6.980974165, 6.821401985),ArrayBuffer(5.659282305, 5.839603903, 5.568601779),0)), (31,(ArrayBuffer(4.462004034, 3.784548524, 3.621580406, 3.258433463, 3.97468681, 6.442817084, 6.329111434, 5.895986022, 6.070395932, 6.61244443),ArrayBuffer(4.829983506, 4.67189283, 4.569898994),0)), (45,(ArrayBuffer(4.103582125, 5.396418618, 3.19...

## Run one PLS regression

In [ ]:
val inputX = mutable.ArrayBuffer(dataXY.map{ case (id, (x, y)) => (id, x) }:_*)
val inputY = mutable.ArrayBuffer(dataXY.map{ case (id, (x, y)) => y }:_*)

PLS.runPLS[mutable.ArrayBuffer[Double]](inputX, inputY, 2)._2

inputX: scala.collection.mutable.ArrayBuffer[(Int, scala.collection.mutable.ArrayBuffer[Double])] = ArrayBuffer((0,ArrayBuffer(4.058845733, 6.405834969, 4.943772245, 4.191393637, 5.691694039, 6.18161279, 6.182385889, 5.982450545, 6.139507172, 7.42310638)), (1,ArrayBuffer(5.102442757, 4.716437077, 4.692925255, 4.839302826, 4.02386069, 2.158591158, 2.901072862, 2.816463237, 3.501312504, 3.084936398)), (2,ArrayBuffer(4.766958169, 4.838583906, 6.485887384, 4.404467358, 5.211054103, 4.865929818, 5.278244008, 4.604447037, 4.98848043, 4.768183695)), (3,ArrayBuffer(6.138171718, 4.494158628, 3.410581785, 5.131981446, 5.551559701, 4.294727934, 5.917064075, 5.219849604, 5.133860522, 4.03092709)), (4,ArrayBuffer(5.375353406, 4.35623841, 4.635715716, 5.089608082, 3.678189181, 2.518132309, 4.75449419...

0.06979240528901356 0.06789620944460527 0.06930445371520205 
0.08067757263085021 0.07970448849559086 0.07974774446651722 
-0.07847738832689169 -0.08644739058771864 -0.0748970917978408 
0.08511456892838887 0.08420743469638495 0.08409775190792484 
0.09009642524949013 0.08958130237869746 0.08888651763257115 
0.14068789390390152 0.1435177674030875 0.13770796553300962 
0.13098330500261812 0.13316536912040206 0.1283447562823357 
8.524690943149948E-5 -0.003144389803075201 0.0010531588356896757 
0.11855232562338051 0.12009478549140792 0.11629398491749238 
0.10618779161310123 0.10710533264884624 0.10430424557462933